TASK:
Load NYC taxi data to DataLake/Blob_Storage/DataBricks and extract the data through dataframe in the notebook.
Perform Following Queries using PySpark.

In [0]:
from pyspark.sql.functions import *

In [0]:
# File location and type
file_location = "/FileStore/tables/yellow_tripdata_2018_01__1_.parquet"
file_type = "parquet"

df = spark.read.format(file_type) \
  .option("sep", ";") \
  .load(file_location)


In [0]:
df.limit(10).display()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2018-01-01T00:21:05.000+0000,2018-01-01T00:24:23.000+0000,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null,null
1,2018-01-01T00:44:55.000+0000,2018-01-01T01:03:05.000+0000,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.0,0.0,0.3,15.3,null,null
1,2018-01-01T00:08:26.000+0000,2018-01-01T00:14:21.000+0000,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.0,0.0,0.3,8.3,null,null
1,2018-01-01T00:20:22.000+0000,2018-01-01T00:52:51.000+0000,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.0,0.0,0.3,34.8,null,null
1,2018-01-01T00:09:18.000+0000,2018-01-01T00:27:06.000+0000,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55,null,null
1,2018-01-01T00:29:29.000+0000,2018-01-01T00:32:48.000+0000,3,0.5,1,N,143,143,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null,null
1,2018-01-01T00:38:08.000+0000,2018-01-01T00:48:24.000+0000,2,1.7,1,N,50,239,1,9.0,0.5,0.5,2.05,0.0,0.3,12.35,null,null
1,2018-01-01T00:49:29.000+0000,2018-01-01T00:51:53.000+0000,1,0.7,1,N,239,238,1,4.0,0.5,0.5,1.0,0.0,0.3,6.3,null,null
1,2018-01-01T00:56:38.000+0000,2018-01-01T01:01:05.000+0000,1,1.0,1,N,238,24,1,5.5,0.5,0.5,1.7,0.0,0.3,8.5,null,null
1,2018-01-01T00:17:04.000+0000,2018-01-01T00:22:24.000+0000,1,0.7,1,N,170,170,2,5.5,0.5,0.5,0.0,0.0,0.3,6.8,null,null


Query 1. - Add a column named as "Revenue" into dataframe which is the sum of the below columns 'Fare_amount','Extra','MTA_tax','Improvement_surcharge','Tip_amount','Tolls_amount','Total_amount'

In [0]:

from pyspark.sql import functions as F
df = df.withColumn(
    "Revenue", 
     F.col("Extra") + F.col("MTA_tax") + F.col("Improvement_surcharge")+F.col('Tip_amount')+F.col('Tolls_amount')+F.col('Total_amount')
)

In [0]:
df.limit(5).display()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Revenue
1,2018-01-01T00:21:05.000+0000,2018-01-01T00:24:23.000+0000,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null,null,7.1
1,2018-01-01T00:44:55.000+0000,2018-01-01T01:03:05.000+0000,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.0,0.0,0.3,15.3,null,null,16.6
1,2018-01-01T00:08:26.000+0000,2018-01-01T00:14:21.000+0000,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.0,0.0,0.3,8.3,null,null,10.600000000000001
1,2018-01-01T00:20:22.000+0000,2018-01-01T00:52:51.000+0000,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.0,0.0,0.3,34.8,null,null,36.099999999999994
1,2018-01-01T00:09:18.000+0000,2018-01-01T00:27:06.000+0000,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55,null,null,20.6


Query 2. - Increasing count of total passengers in New York City by area

In [0]:
df.groupBy("PULocationID",'DOLocationID').sum("passenger_count").limit(5).display()

PULocationID,DOLocationID,sum(passenger_count)
79,116,580
246,249,5073
234,144,7047
161,193,100
231,261,5812


Query 3. - Realtime Average fare/total earning amount earned by 2 vendors

In [0]:
df.groupBy("VendorID").avg("total_amount").display()

VendorID,avg(total_amount)
1,15.127384289902137
2,15.775723474073514


Query 4. - Moving Count of payments made by each payment mode

In [0]:
df.groupBy('payment_type'). count().display()

payment_type,count
1,6106416
3,43204
2,2599215
4,11852


Query 5. - Highest two gaining vendor's on a particular date with no of passenger and total distance by cab

In [0]:
df.select("VendorID","tpep_pickup_datetime","passenger_count","trip_distance").sort(df.trip_distance.desc(),df.passenger_count.desc()).limit(2).display()

VendorID,tpep_pickup_datetime,passenger_count,trip_distance
2,2018-01-30T11:41:02.000+0000,1,189483.84
1,2018-01-08T19:44:54.000+0000,0,830.8


Query 6. - Most no of passenger between a route of two location.

In [0]:
query = (df.groupBy("PULocationID",'DOLocationID').agg(sum("passenger_count").alias("Most_no_of_passenger")))
query_f = query.orderBy(col('Most_no_of_passenger').desc())
display(query_f.limit(1))

PULocationID,DOLocationID,Most_no_of_passenger
264,264,186705


Query 7. - Get top pickup locations with most passengers in last 5/10 seconds.

In [0]:
Query=(df.groupBy(window("tpep_pickup_datetime","5 seconds"))
   .agg(sum("passenger_count").alias("sum_passenger"),count("PULocationID").alias("Top_pickup")))

Query_f= Query.orderBy(col('Top_pickup').desc())
display(Query_f.limit(1))

window,sum_passenger,Top_pickup
"List(2018-01-27T15:15:25.000+0000, 2018-01-27T15:15:30.000+0000)",74,64
